In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import statsmodels.api as sm 
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import pprint as pp
import itertools
from collections import ChainMap
from sklearn.model_selection import cross_validate
import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_hdf('Data/preprocessed/df_ml_hdf.h5', key='match')
df.dropna(inplace=True)

def preselect_non_redundant_features(df, target, corr_threshold):
    corr = df.corr()
    corr[target] = abs(corr[target])
    corr = corr.sort_values(target, ascending=False)

    #prepare lower trainagular matrix (with deleted diagonal)
    low_tri_no_diag = np.tril(np.ones(corr.shape).astype(np.bool), k=-1)
    corr = corr.where(low_tri_no_diag)
    #drop tagret variable form rows and columns
    corr = corr.drop(corr.columns[0], axis=1).drop(corr.index[0], axis=0)

    corr['MaxCorr'] = corr.abs().max(axis = 1)

    pre_selected_features = corr[corr['MaxCorr'] < corr_threshold].index 
    return pre_selected_features
    
def get_non_redundant_df(df, target, corr_threshold):
    features = preselect_non_redundant_features(df, target, corr_threshold)
    output = df.loc[:,features]
    return output


def feature_based_train_test_split(df, target, splitting_feature, test_split_value, 
                                   splitting_feature_as_index=True):
    if splitting_feature_as_index == True:
        X_train = df[~df.index.isin([test_split_value], level = splitting_feature)]
        X_test = df[df.index.isin([test_split_value], level = splitting_feature)]
                                  
        y_train = df[~df.index.isin([test_split_value], level = splitting_feature)].reset_index()[target].values
        y_test = df[df.index.isin([test_split_value], level = splitting_feature)].reset_index()[target].values
                   
    else:
        X_train = df[~df[splitting_feature]==test_split_value]
        X_test = df[df[splitting_feature]==test_split_value]
        
        y_train = df[~df[splitting_feature]==test_split_value].reset_index()[target].values
        y_test = df[df[splitting_feature]==test_split_value].reset_index()[target].values
        
    return X_train, y_train, X_test, y_test

  
#df = get_non_redundant_df(df, 'PTS_Result', 0.5) 
       
X_train, y_train, X_test, y_test = feature_based_train_test_split(df, 'Won_Result', 'season', '2019-20')    
    

In [3]:
df.columns = df.columns.to_flat_index()
df.head()

(Last_Game_Res, )  \
Team season  Game Number Match_id           Match_Team_id          Home/Away Opponent Rivalry Won_Result Game Date                       
BKN  2013-14 34.0        ATL_BKN_01/06/2014 BKN_ATL_BKN_01/06/2014 Home      ATL      ATL_BKN True       2014-01-06                1.0   
     2018-19 43.0        ATL_BKN_01/09/2019 BKN_ATL_BKN_01/09/2019 Home      ATL      ATL_BKN True       2019-01-09                0.0   
     2016-17 37.0        ATL_BKN_01/10/2017 BKN_ATL_BKN_01/10/2017 Home      ATL      ATL_BKN False      2017-01-10                1.0   
ATL  2017-18 42.0        ATL_BKN_01/12/2018 ATL_ATL_BKN_01/12/2018 Home      BKN      ATL_BKN False      2018-01-12                0.0   
BKN  2019-20 38.0        ATL_BKN_01/12/2020 BKN_ATL_BKN_01/12/2020 Home      ATL      ATL_BKN True       2020-01-12                0.0   

                                                                                                                     (PTS_Result, )  \
Team season  Game Number Match_id           Match_Team_id          Home/Away Opponent Rivalry Won_Result Game Date                    
BKN  2013-14 34.0        ATL_BKN_01/06/2014 BKN_ATL_BKN_01/06/2014 Home      ATL      ATL_BKN True       2014-01-06               5   
     2018-19 43.0        ATL_BKN_01/09/2019 BKN_ATL_BKN_01/09/2019 Home      ATL      ATL_BKN True       2019-01-09              16   
     2016-17 37.0        ATL_BKN_01/10/2017 BKN_ATL_BKN_01/10/2017 Home      ATL      ATL_BKN False      2017-01-10             -20   
ATL  2017-18 42.0        ATL_BKN_01/12/2018 ATL_ATL_BKN_01/12/2018 Home      BKN      ATL_BKN False      2018-01-12              -5   
BKN  2019-20 38.0        ATL_BKN_01/12/2020 BKN_ATL_BKN_01/12/2020 Home      ATL      ATL_BKN True       2020-01-12              22   

                                                                                                                     (ewm_0.3_ssn_team, MIN)  \
Team season  Game Number Match_id           Match_Team_id          Home/Away Opponent Rivalry Won_Result Game Date                             
BKN  2013-14 34.0        ATL_BKN_01/06/2014 BKN_ATL_BKN_01/06/2014 Home      ATL      ATL_BKN True       2014-01-06                      0.0   
     2018-19 43.0        ATL_BKN_01/09/2019 BKN_ATL_BKN_01/09/2019 Home      ATL      ATL_BKN True       2019-01-09                      0.0   
     2016-17 37.0        ATL_BKN_01/10/2017 BKN_ATL_BKN_01/10/2017 Home      ATL      ATL_BKN False      2017-01-10                      0.0   
ATL  2017-18 42.0        ATL_BKN_01/12/2018 ATL_ATL_BKN_01/12/2018 Home      BKN      ATL_BKN False      2018-01-12                      0.0   
BKN  2019-20 38.0        ATL_BKN_01/12/2020 BKN_ATL_BKN_01/12/2020 Home      ATL      ATL_BKN True       2020-01-12                      0.0   

                                                                                                                     (ewm_0.3_ssn_team, OffRtg)  \
Team season  Game Number Match_id           Match_Team_id          Home/Away Opponent Rivalry Won_Result Game Date                                
BKN  2013-14 34.0        ATL_BKN_01/06/2014 BKN_ATL_BKN_01/06/2014 Home      ATL      ATL_BKN True       2014-01-06                        11.6   
     2018-19 43.0        ATL_BKN_01/09/2019 BKN_ATL_BKN_01/09/2019 Home      ATL      ATL_BKN True       2019-01-09                        -6.6   
     2016-17 37.0        ATL_BKN_01/10/2017 BKN_ATL_BKN_01/10/2017 Home      ATL      ATL_BKN False      2017-01-10                       -11.0   
ATL  2017-18 42.0        ATL_BKN_01/12/2018 ATL_ATL_BKN_01/12/2018 Home      BKN      ATL_BKN False      2018-01-12                        32.6   
BKN  2019-20 38.0        ATL_BKN_01/12/2020 BKN_ATL_BKN_01/12/2020 Home      ATL      ATL_BKN True       2020-01-12                        24.7   

                                                                                                                     (ewm_0.3_ssn_team, DefRtg)  \
Team season  Game 

In [15]:
scaler = [StandardScaler(), MinMaxScaler(), 'passthrough']
scaler_dict = {'scaler':scaler}

pca_parameters = {PCA : {'n_components':[3], 'passthrough':True}}
pca = create_pipe_grid(pca_parameters)
pca_dict = {'PCA':pca}

selector_parameters = {SelectKBest: {'k':[5,10], 'score_func':[f_classif], 'passthrough':False}}
selector = create_pipe_grid(selector_parameters)
selector_dict = {'selector': selector}

'''
clf_parameters = {LogisticRegression:{'C':[0.1, 1, 10], 'penalty':['none', 'l2']},
              RandomForestClassifier:{'max_depth':[2,5,10,None]},
              SVC:{'C':[0.1,1,10]},
              AdaBoostClassifier:{'n_estimators':[20,50,100]},
              GaussianProcessClassifier:{'max_iter_predict':[50,100]}
              }
'''
clf_parameters = {LogisticRegression:{'C':[1, 10], 'penalty':['l2']}}

clf = create_pipe_grid(clf_parameters)
clf_dict = {"clf":clf}

NameError: name 'MinMaxScaler' is not defined

In [18]:
type(selector_dict['selector'][0])

sklearn.feature_selection._univariate_selection.SelectKBest

In [9]:
def create_pipe_grid(func_parameters):
    '''
    Parameters
    ----------
    func_parameters : dict
        dictionary in form of: 
        {sklearn method : {param_name1 : [param_val1, param_val2..], param_name2: [param_val1, param_val2], 'passthrough':True}}
    'passthrough' key is optional. If its set to True 'passthrough' for sklearn Pipeline will be added to specific step
    Returns
    multi_grid: list
        List of arguments (invoked sklearn methods with specific parameters) to be searched through by pipeline
    -------
    '''
    
    multi_grid = []
    for func, parameters_dict in func_parameters.items():
            
        main_list=[]
        passthrough = False
        
        for key in list(parameters_dict.keys()):
            
            if key == 'passthrough':
                if parameters_dict[key] == True: 
                    passthrough = True
                else: pass
        
            else: 
                dicts_list=[]
                for i in range(len(list(parameters_dict[key]))):
                    param_dict = {key:list(parameters_dict[key])[i]}
                    dicts_list.append(param_dict)
                main_list.append(dicts_list)
        
        param_variant = list(itertools.product(*main_list))
        
        
        param_variants = [dict(ChainMap(*param_variant[i])) for i in range(len(param_variant))]
        
        grid = []
        for variant in param_variants:
            grid.append(func(**variant))
       
        #add passthrough keyword if it appeared as argument in dictionary and value was True
        if passthrough == True:
            grid.append('passthrough')
        
        multi_grid.extend(grid)
        
    return(multi_grid)

In [11]:
def pipeline_search_report(X_train, y_train, X_test, y_test, data_description, *kwargs):
    
    steps_lists = [list(arg.values())[0] for arg in kwargs]
    step_names = [list(arg.keys())[0] for arg in kwargs] 
    results = []
    
    for pipe_steps in itertools.product(*steps_lists):
        steps = list(zip(step_names, pipe_steps)) 
        pipe = Pipeline(steps)
        pipe.fit(X_train, y_train)
        cv = cross_validate(pipe, X_train, y_train, cv=5)
        cv_scores = [np.mean(cv[key]) for key in cv.keys()]
        test_score = pipe.score(X_test, y_test)
        timestamp = datetime.datetime.now()
        results.append([timestamp, data_description, *pipe_steps, *cv_scores, test_score])
    
    cv_score_types = ['cv_' + k for k in cv.keys()]
    col_names = ['Timestamp', 'Data Description', *step_names, *cv_score_types, 'test_score']
    
    result_df = pd.DataFrame(results, columns=col_names)
    result_df = result_df.sort_values(by=['test_score'], ascending=False)
    return result_df



In [12]:
def cluster_data(X_train, y_train, X_test, y_test, k=2, pca_components=3):
    
    X_train_sc, X_test_sc = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)
    
    pca = PCA(n_components=pca_components)
    pca_comps_train = pca.fit_transform(X_train_sc)
    pca_comps_test = pca.fit_transform(X_test_sc)
    
    pca_comps_train_df = pd.DataFrame(pca_comps_train)
    pca_comps_test_df = pd.DataFrame(pca_comps_test)
    
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(pca_comps_train_df)
    cluster_labels_train = kmeans.predict(pca_comps_train_df)
    cluster_labels_test = kmeans.predict(pca_comps_test_df)
    
    cluster_data = {}
    for k in range(k):
        X_train_k = X_train[cluster_labels_train == k]
        y_train_k = y_train[cluster_labels_train == k]
        X_test_k = X_test[cluster_labels_test == k]
        y_test_k = y_test[cluster_labels_test == k]
        
        data_k = [X_train_k, y_train_k, X_test_k, y_test_k]
        cluster_data[k] = data_k
    
    return cluster_data

In [13]:
def cluster_pipeline_search_report(clustered_data_dict, data_description, *kwargs):
    
    steps_lists = [list(arg.values())[0] for arg in kwargs]
    step_names = [list(arg.keys())[0] for arg in kwargs] 
    results = []
    for k in clustered_data_dict.keys():
        X_train = clustered_data_dict[k][0]
        y_train = clustered_data_dict[k][1]
        X_test = clustered_data_dict[k][2]
        y_test = clustered_data_dict[k][3]
        for pipe_steps in itertools.product(*steps_lists):
            steps = list(zip(step_names, pipe_steps)) 
            pipe = Pipeline(steps)
            pipe.fit(X_train, y_train)
            cv = cross_validate(pipe, X_train, y_train, cv=5)
            cv_scores = [np.mean(cv[key]) for key in cv.keys()]
            test_score = pipe.score(X_test, y_test)
            timestamp = datetime.datetime.now()
            results.append([timestamp, k, data_description, *pipe_steps, *cv_scores, test_score])

    cv_score_types = ['cv_' + k for k in cv.keys()]
    col_names = ['Timestamp', 'Cluster', 'Data Description', *step_names, *cv_score_types, 'test_score']
    
    result_df = pd.DataFrame(results, columns=col_names)
    result_df = result_df.sort_values(by=['test_score'], ascending=False)
    return result_df

In [14]:
clustered_data = cluster_data(X_train, y_train, X_test, y_test, k=2, pca_components=3)
result_report = cluster_pipeline_search_report(clustered_data, "2 clusters, 19-20 season as test", scaler_dict, selector_dict, clf_dict)

C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mateu\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [ ]:
result_report[result_report['Cluster']==1].head()